In [3]:
# 데이터 확인 및 정리
print("격자예보 데이터 정보:")
print(f"총 행 수: {len(df_grid)}")
print(f"위도 결측값: {df_grid[lat1].isna().sum()}")
print(f"경도 결측값: {df_grid[lon1].isna().sum()}")
print(f"위도 범위: {df_grid[lat1].min()} ~ {df_grid[lat1].max()}")
print(f"경도 범위: {df_grid[lon1].min()} ~ {df_grid[lon1].max()}")

print("\n관측지점 데이터 정보:")
print(f"총 행 수: {len(df_meta)}")
print(f"위도 결측값: {df_meta[lat2].isna().sum()}")
print(f"경도 결측값: {df_meta[lon2].isna().sum()}")
print(f"위도 범위: {df_meta[lat2].min()} ~ {df_meta[lat2].max()}")
print(f"경도 범위: {df_meta[lon2].min()} ~ {df_meta[lon2].max()}")

# 결측값 및 무한값 제거
print("\n데이터 정리 중...")
df_meta_clean = df_meta.dropna(subset=[lat2, lon2])
df_meta_clean = df_meta_clean[
    np.isfinite(df_meta_clean[lat2]) & 
    np.isfinite(df_meta_clean[lon2])
]

df_grid_clean = df_grid.dropna(subset=[lat1, lon1])
df_grid_clean = df_grid_clean[
    np.isfinite(df_grid_clean[lat1]) & 
    np.isfinite(df_grid_clean[lon1])
]

print(f"정리 후 관측지점 수: {len(df_meta_clean)}")
print(f"정리 후 격자예보 수: {len(df_grid_clean)}")

격자예보 데이터 정보:
총 행 수: 3831
위도 결측값: 0
경도 결측값: 0
위도 범위: 0.0 ~ 38.4916444444444
경도 범위: 0.0 ~ 131.8648471

관측지점 데이터 정보:
총 행 수: 6515
위도 결측값: 9
경도 결측값: 9
위도 범위: 33.1137 ~ 38.5765
경도 범위: 124.6305 ~ 131.8714

데이터 정리 중...
정리 후 관측지점 수: 6506
정리 후 격자예보 수: 3831


In [5]:
# 정리된 데이터로 KDTree 생성 및 매핑
if len(df_meta_clean) == 0:
    print("ERROR: 관측지점 데이터에 유효한 좌표가 없습니다!")
elif len(df_grid_clean) == 0:
    print("ERROR: 격자예보 데이터에 유효한 좌표가 없습니다!")
else:
    # 관측지점정보의 위경도 배열로 KDTree 생성
    meta_coords_clean = df_meta_clean[[lat2, lon2]].to_numpy()
    tree = cKDTree(meta_coords_clean)
    
    # 격자예보 각 행에 대해 최근접 관측지점 인덱스 찾기
    grid_coords_clean = df_grid_clean[[lat1, lon1]].to_numpy()
    dist, idx = tree.query(grid_coords_clean, k=1)
    
    # 결과를 원본 df_grid에 매핑하기 위해 인덱스 재조정
    df_result = df_grid_clean.copy()
    df_result["관측지점_매핑인덱스"] = idx
    df_result["매핑거리(km)"] = dist * 111  # 대략적인 km 변환 (위도 1도 ≈ 111km)
    
    # 매핑거리가 5km 이하인 데이터만 필터링
    df_result = df_result[df_result["매핑거리(km)"] <= 5.0]
    print(f"5km 이내 매핑된 지점 수: {len(df_result)}")
    
    if len(df_result) == 0:
        print("ERROR: 5km 이내에 매핑된 지점이 없습니다!")
    else:
        # 필터링된 인덱스로 다시 매핑
        filtered_idx = df_result["관측지점_매핑인덱스"].values
        
        # 최근접 관측지점의 정보 붙이기
        for col in df_meta_clean.columns:
            matched_data = df_meta_clean.iloc[filtered_idx][col].values
            df_result[f"관측_{col}"] = matched_data
    
    # 필요한 컬럼만 선택하여 최종 결과 생성
    final_columns = ["1단계", "2단계", "3단계", "격자 X", "격자 Y", "위도", "경도", "관측_노장해발고도(m)"]
    
    # 컬럼명 매핑 (관측지점정보에서 가져온 컬럼명에 맞춰 조정)
    available_columns = []
    for col in final_columns:
        if col in df_result.columns:
            available_columns.append(col)
        elif col == "관측_노장해발고도(m)" and "관측_해발고도(m)" in df_result.columns:
            # 노장해발고도와 해발고도가 다를 수 있으므로 확인
            available_columns.append("관측_해발고도(m)")
            print("주의: '노장해발고도(m)' 대신 '해발고도(m)' 컬럼을 사용합니다.")
    
    if available_columns:
        df_final = df_result[available_columns].copy()
        
        # 컬럼명에서 "관측_" 접두사 제거
        df_final.columns = [col.replace("관측_", "") if col.startswith("관측_") else col for col in df_final.columns]
        
        # 결과 저장
        df_final.to_csv("격자예보_관측지점매핑.csv", index=False, encoding="utf-8-sig")
        print(f"매핑 완료! 총 {len(df_final)}개 지점이 매핑되었습니다.")
        print("'격자예보_관측지점매핑.csv' 파일을 확인하세요.")
        print(f"최종 파일 컬럼: {list(df_final.columns)}")
        print(f"\n매핑거리 통계:")
        print(f"최소 매핑 거리: {df_result['매핑거리(km)'].min():.3f} km")
        print(f"평균 매핑 거리: {df_result['매핑거리(km)'].mean():.3f} km")
        print(f"최대 매핑 거리: {df_result['매핑거리(km)'].max():.3f} km")
    else:
        print("ERROR: 요청된 컬럼들을 찾을 수 없습니다.")
        print(f"사용 가능한 컬럼: {list(df_result.columns)}")

5km 이내 매핑된 지점 수: 3809
매핑 완료! 총 3809개 지점이 매핑되었습니다.
'격자예보_관측지점매핑.csv' 파일을 확인하세요.
최종 파일 컬럼: ['1단계', '2단계', '3단계', '격자 X', '격자 Y', '위도', '경도', '노장해발고도(m)']

매핑거리 통계:
최소 매핑 거리: 0.001 km
평균 매핑 거리: 0.820 km
최대 매핑 거리: 4.855 km
